<a href="https://colab.research.google.com/github/CP2J/cp2j/blob/ACJ-8-SVD/Recsys_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # SVD(Singular Value Decomposition)

 특잇값 분해를 통해 차원축소를 진행 - sparse 값 해결
 

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
files.upload();

Saving movies.csv to movies.csv
Saving ratings.csv to ratings.csv
Saving users.csv to users.csv


In [ ]:
rating = pd.read_csv('ratings.csv')
rating.head()

,user_id,item_id,rating,timestamp
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [ ]:
rating_matrix = rating.pivot(index = 'user_id', columns = 'item_id', values= 'rating')
print(rating_matrix.shape)
rating_matrix.head(5)

(943, 1682)


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import copy

In [ ]:
rating_matrix_copy = rating_matrix.copy()
for i in range(rating_matrix.shape[0]):
  rating_matrix.iloc[i] = rating_matrix_copy.iloc[i].fillna(rating_matrix.iloc[i].mean())

In [ ]:
rating_matrix.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.000000,3.000000,4.000000,3.000000,3.000000,5.000000,4.000000,1.000000,5.000000,3.000000,...,3.610294,3.610294,3.610294,3.610294,3.610294,3.610294,3.610294,3.610294,3.610294,3.610294
2,4.000000,3.709677,3.709677,3.709677,3.709677,3.709677,3.709677,3.709677,3.709677,2.000000,...,3.709677,3.709677,3.709677,3.709677,3.709677,3.709677,3.709677,3.709677,3.709677,3.709677
3,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296,...,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296,2.796296
4,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,...,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333,4.333333
5,4.000000,3.000000,2.874286,2.874286,2.874286,2.874286,2.874286,2.874286,2.874286,2.874286,...,2.874286,2.874286,2.874286,2.874286,2.874286,2.874286,2.874286,2.874286,2.874286,2.874286


In [ ]:
from numpy.linalg import svd

In [ ]:
U, Sigma, Vt = svd(rating_matrix)

In [ ]:
print(U.shape) # 왼쪽 singular matrix 열벡터
print(Sigma.shape) # 대각행렬 특이값들, 이 중 상위값을 사용하여 차원 축소
print(Vt.shape) # 오른쪽 singular matrix 열벡터
# U * Singma * Vt = full SVD

(943, 943)
(943,)
(1682, 1682)


In [ ]:
Sigma[:10]

array([4553.93679047,   65.84787761,   50.46197457,   38.69420261,
         37.16956009,   35.95477384,   33.92771003,   32.78337827,
         30.83984104,   29.5307084 ])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 평점 데이터 나누기
train, test = train_test_split(rating, test_size = 0.3)

In [ ]:
def svd_function(df, k = 30):
  # 평점 테이블 피벗 후 svd, k차원으로 축소
  df_pivot = df.pivot(index = 'user_id', columns = 'item_id', values = 'rating')
  df_pivot_c = df_pivot.copy()
  for i in range(df_pivot.shape[0]):
    df_pivot.iloc[i] = df_pivot_c.iloc[i].fillna(df_pivot_c.iloc[0].mean())
  U, Sigma, Vt = svd(df_pivot)
  U = U[:, :k] # 열 k개 사용
  Sigma = np.diag(Sigma[:k]) # 상위값 k 개 대각행렬
  Vt = Vt[:k] # 행 k개 사용
  #원래 m*n = m*s * s*s * s*n 이었던 값을
  #    m*n = m*k * k*k * k*n 으로 축소 (s>k)
  
  # 행렬 곱 진행
  Usig = np.dot(U, Sigma)
  Usigvt = np.dot(Usig, Vt)
  svd_df = pd.DataFrame(Usigvt,index = df_pivot.index, columns = df_pivot.columns)
  return svd_df

In [ ]:
result_train = svd_function(train)
result_train # pivot한 train set 차원축소 후 행렬 곱 결과

item_id,1,2,3,4,5,6,7,8,9,10,...,1670,1671,1672,1673,1674,1675,1678,1679,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,4.023405,3.706920,3.522804,4.039452,3.315952,3.768863,4.774394,3.980051,4.051484,3.625421,...,3.624445,3.623430,3.550270,3.626993,3.626455,3.624597,3.610426,3.621069,3.594673,3.618582
2,3.838006,3.571883,3.643707,3.645418,3.611877,3.707254,3.709158,3.709829,3.702830,3.620251,...,3.652699,3.645523,3.655627,3.651401,3.652356,3.650814,3.639840,3.649320,3.654049,3.652137
3,3.405812,3.705752,3.695508,3.681521,3.610308,3.593087,3.429025,3.593279,3.730523,3.713153,...,3.652186,3.652857,3.656857,3.653915,3.653983,3.653677,3.660916,3.654568,3.657220,3.656365
4,3.904734,3.701615,3.652603,3.572718,3.674730,3.677817,3.815885,3.741965,3.701066,3.694224,...,3.659475,3.658235,3.634697,3.660575,3.658172,3.657549,3.645240,3.654976,3.650395,3.657094
5,3.751993,2.902585,2.844449,3.963183,3.668023,3.575905,3.641700,4.134087,3.650384,3.640446,...,3.658236,3.664755,3.693472,3.654787,3.641614,3.640436,3.655744,3.646947,3.660270,3.654287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,3.804546,3.666803,3.625311,3.558064,3.793299,3.613046,4.065170,3.634405,3.504112,3.696794,...,3.660411,3.656910,3.666066,3.663820,3.665332,3.664340,3.645065,3.660315,3.666748,3.668645
940,3.860304,3.668392,3.664957,3.287859,3.606204,3.672845,4.171230,3.771924,3.595408,3.593815,...,3.641016,3.633307,3.619983,3.647694,3.648588,3.650113,3.631868,3.644384,3.638540,3.649972
941,3.773104,3.635146,3.645925,3.753533,3.647168,3.678702,3.790392,3.668708,3.736978,3.693736,...,3.657535,3.654787,3.647996,3.657502,3.657131,3.656150,3.650764,3.655717,3.654773,3.656859


In [ ]:
result_train.isnull().sum()

item_id
1       0
2       0
3       0
4       0
5       0
       ..
1675    0
1678    0
1679    0
1681    0
1682    0
Length: 1629, dtype: int64

 평가 함수

In [ ]:
df = pd.DataFrame([[1, 1.5], [2, 2.5]], columns=['int', 'float'])
list(df.iterrows())[1]

(1,
 int      2.0
 float    2.5
 Name: 1, dtype: float64)

In [ ]:
def eval_function(result_train, train, test):
  result = []
  # test에서 행 별로 뽑아 행 데이터 사용
  for index, row in test.iterrows():
    user = row['user_id']
    item = row['item_id']
    actual_rating = row['rating']
    #1. 예측할 test의 유저와 item이 둘다 예측 셋인 result_train 내 있는 경우 
    # result_train 내 예측값 가져오기
    if user in result_train.index.values and item in result_train.columns.values:
      pred_rating = result_train.loc[user][item]
    #2. train에 유저는 있는데 item이 없었을 경우 (train 시 아무도 이 영화를 안봤음)
    # - result_train내 user의 다른 item들의 평점 평균으로 한다.
    elif user in result_train.index.values:
      pred_rating = result_train.loc[user].mean()
    #3. train에 아이템은 있는데 user가 없었을 경우 (train 에 user 평점들이 아예 안들어감)
    # - result_train내 item에 대한 다른 사람들의 평점 평균으로 한다.      
    elif item in result_train.index.values:
      pred_rating = result_train[item].mean()
    #4. train에 둘 다 없음(train 셋 내 이 영화 본 사람도 없고 user의 평점도 없음)
    # test셋 내 user의 평점 평균
    else:
      pred_rating = test[test['user_id']==user]['rating'].mean()
    result.append([user, item, actual_rating, pred_rating])
  # 한 줄 씩 추가한 내용들 데이터프레임화 (유저, 영화, 실제 평점, 예상 평점)
  df_result = pd.DataFrame(result, columns = ['user', 'item', 'actual_rating', 'pred_rating'])
  # rmse 값 (root mean squared error)
  return np.sqrt(np.square(df_result['pred_rating']-df_result['actual_rating']).mean())


In [ ]:
eval_function(result_train, train, test)

1.0685257539582291